# Time Series Forecasting

Time series forecasting is the use of a model to predict future values based on previously observed values. 

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

### Data Preparation

In [ ]:
# read and process the charts dataset
df = pd.read_csv('data/spotify_daily_charts.csv')
#transform date column into a datetime column
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
df.head()

## 1. Error metrics

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred): 
    return np.sqrt(mean_squared_error(y_true, y_pred))

def show_errors(y_true, y_pred):
    return pd.DataFrame({'RMSE': root_mean_squared_error(y_true,y_pred),\
                                            'MAE':mean_absolute_error(y_true,y_pred),\
                                            'MAPE': mean_absolute_percentage_error(y_true,y_pred)}, index=[0])

The **Root Mean Square Error (RMSE)** penalizes larger errors higher, while the **Mean Absolute Error (MAE, meanabs)** weighs all errors equally regardless of magnitude. The **Mean Absolute Percentage Error** expresses the error as a percentage by weighting the MAE by the actual value.

In [ ]:
test_actual = [2,4,6]
test_forecast = [1,2,3]
show_errors(test_actual,test_forecast)

## 2. Forecasting methods

Here are but a few techniques that could help you start exploring this field.

>Q: Can you predict Ben&Ben's Kathang Isip daily streams for the Oct to Nov 2021 using data from past dates?

In [ ]:
data = df[(df.index.year>=2019)&(df['track_name']=='Kathang Isip')][['streams']]

In [ ]:
#convert streams to hundred thousands
data = data/100000

For simplicity, we conduct a holdout to split the time series data into 1 train and 1 test set. 
 - The test set will be the period of interest (Oct-Nov 2021)
 - We limit the train set to only up to Jun 2021 (1 quarter back)
 
Furthermore, we assume that the test set is *streamed* on a daily basis, i.e. new data comes in daily and can be used for next day's forecast.

In [ ]:
#split dataset to training and test sets
train_df = data['2021-06-01':'2021-09-30']
test_df = data['2021-10-01':'2021-11-30']

fig = plt.figure(figsize=(15,3))
plt.plot(train_df['streams'], color='C0', label='train')
plt.plot(test_df['streams'], color='C7', label='test')
plt.legend()
plt.ylabel("Streams x 100K")

### 2.1. Previous value (Naive) Approach
The simplest forecast--the previous available value is the forecast value.

In [ ]:
# To simulate a naive forecast that takes in incoming data from test set
# shift the data 1 day to the right
forecast_df = data.shift(1)['2021-10-01':'2021-11-30']
forecast_df

In [ ]:
# plot the forecast
fig = plt.figure(figsize=(15,3))
plt.plot(train_df['streams'], color='C0', label='train')
plt.plot(test_df['streams'], color='C7', label='test')
plt.plot(forecast_df['streams'], color='C1', label='forecast')
plt.legend()
plt.ylabel("Streams x 100K")

In [ ]:
show_errors(test_df['streams'].values,forecast_df['streams'].values)

Modification: Previous-week (D-7) value

In [ ]:
# shift the data 7 day to the right
forecast_df = data.shift(7)['2021-10-01':'2021-11-30']

In [ ]:
# plot the forecast
fig = plt.figure(figsize=(15,3))
plt.plot(train_df['streams'], color='C0', label='train')
plt.plot(test_df['streams'], color='C7', label='test')
plt.plot(forecast_df['streams'], color='C1', label='forecast')
plt.legend()
plt.ylabel("Streams x 100K")

In [ ]:
show_errors(test_df['streams'],forecast_df['streams'])

### 2.2. Windowed Average Approach
The average over the X recent days is the forecast value

In [ ]:
# To simulate a windowed forecast that takes in incoming data from test set
# use the pd.rolling grouper and pass the mean method
# and shift 1 day ahead to match target forecast date

#Set window to 7 days
forecast_df = data.rolling(7).mean().shift(1)['2021-10-01':'2021-11-30']

In [ ]:
# plot the forecast
fig = plt.figure(figsize=(15,3))
plt.plot(train_df['streams'], color='C0', label='train')
plt.plot(test_df['streams'], color='C7', label='test')
plt.plot(forecast_df['streams'], color='C1', label='forecast')
plt.legend()
plt.ylabel("Streams x 100K")

In [ ]:
show_errors(test_df['streams'],forecast_df['streams'])

### 2.3. Exponential Average Approach
The weighted average over the most recent days is the forecast value, where the weights are given by the following formula as set by a smoothing factor $\alpha$

![](https://miro.medium.com/max/1256/1*gzC8tdlwaLM3Y1tiaM3xzw.png)

 Larger $\alpha$ values will assign larger weights to more recent dates.

In [ ]:
# To simulate an EMA  forecast that takes in incoming data from test set
# use the pd.rolling grouper and pass the mean method
# and shift 1 day ahead to match target forecast date

#Set window to 7 days
#Set alpha to 0.7, i.e. 70% of the forecast will come from the most recent data point

forecast_df = data.ewm(alpha=0.7).mean().shift(1)['2021-10-01':'2021-11-30']

In [ ]:
# plot the forecast
fig = plt.figure(figsize=(15,3))
plt.plot(train_df['streams'], color='C0', label='train')
plt.plot(test_df['streams'], color='C7', label='test')
plt.plot(forecast_df['streams'], color='C1', label='forecast')
plt.legend()
plt.ylabel("Streams x 100K")

In [ ]:
show_errors(test_df['streams'],forecast_df['streams'])

### 2.4. Trend and Seasonality Approaches 
One example of a forecasting method that takes trend and seasonality into account is the Holt-Winters method, which uses exponential averages over components. The **Holt-Winters seasonal method** comprises the forecast equation and three smoothing equations

![](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/01/eq.png)

The method implemented in statsmodels automatically computes for the required parameters

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing

We generate the forecast by calling the fitted model to generate values for every required timestep. 
For this example , we tune the model one-time, and only on the initial train data

In [ ]:
# specify fit for weekly (d=7) seasonal cycles
# assume additive combinations of component
model_fit = ExponentialSmoothing(train_df['streams'],seasonal_periods=7 ,trend='add', seasonal='add').fit()
forecast_df = pd.DataFrame(model_fit.forecast(len(test_df)).values, index=test_df.index,\
                           columns=['streams']) 

In [ ]:
#plot the fitted training data
fig = plt.figure(figsize=(15,3))

plt.plot(train_df['streams'], color='C0', label='train')
plt.plot(test_df['streams'], color='C7', label='test')
plt.plot(forecast_df['streams'], color='C1', label='forecast')
plt.legend()
plt.ylabel("Streams x 100K")

In [ ]:
show_errors(test_df['streams'],forecast_df['streams'])

### 2.5. Stochastic Approaches
A popular and frequently used stochastic (i.e. probabilistic "randomness" based models) time series model is the ARIMA model. It assumes that the time-series is linear and follows a particular known statistical distribution, such as the normal distribution, and has subclass of other models such as the Autoregressive (AR) model, the Moving average (MA) model, and the Autoregressive Moving Average (ARMA) model of which the ARIMA model was based on. 

An *ARIMA model* is usually written as ARIMA (p,d,q) 

![](https://miro.medium.com/max/875/1*J1cOKMRU17nr71T-Xx6_HQ.png)

where: 

p = The order of the Autoregressive part of the model

d= The degree of first differencing in our model

q = The order of the Moving average part of the model

**How do we get the optimal values for these parameters?**

**p**: The PACF can be used to determine how many AR terms you need to use to explain the autocorrelation pattern in a time series: if the partial autocorrelation is significant at lag k and not significant at any higher order lags — i.e., if the PACF “cuts off” at lag k — then this suggests that you should try fitting an autoregressive model of order k

**q**: The ACF can be used to determine how many MA terms you need for a model. Another way to think about the moving average model is that it corrects future forecasts based on errors made on recent forecasts. We would expect the ACF for the MA(k) process to show a strong correlation with recent values up to the lag of k, then a sharp decline to low or no correlation. 

**d**: Used only when time series is not stationary. Usually d=1 or 2 is sufficient

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
#PACF
fig = plt.figure(figsize=(10,6))
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)

acf = plot_acf(train_df['streams'].interpolate().diff()[1:], lags=50, ax=ax1)
pacf = plot_pacf(train_df['streams'].interpolate().diff()[1:], lags=50, ax=ax2)

From the ACF, it seems like the model needs to take in 7 MA terms (i.e. moving averages up to 7 days), i.e. q=7

From the PACF, it is likely that a model with up to 7 AR terms also would be best to use, i.e. p=7

We set d = 1 to stationarize the train set

In [ ]:
model = ARIMA(train_df, order=(7, 1, 7))  
model_fit = model.fit() 
train_fit_df = pd.DataFrame(model_fit.fittedvalues, columns=['streams'])

In [ ]:
#plot the fitted training data
fig = plt.figure(figsize=(15,3))

plt.plot(train_df['streams'], color='C0', label='train')
plt.plot(test_df['streams'], color='C7', label='test')
plt.plot(train_fit_df['streams'], color='C1', label='train_fit')
plt.legend()
plt.ylabel("Streams x 100K")

We generate the forecast by calling the fitted model to generate values for every required timestep. 
For this example , we tune the model one-time, and only on the initial train data

In [ ]:
#dynamic = True toggles use of preceding forecast value with the model fit values to get the next forecast value
forecast_df = pd.DataFrame(model_fit.forecast(len(test_df),dynamic=True).values, index=test_df.index,\
                           columns=['streams'])

In [ ]:
#plot the fitted training data
fig = plt.figure(figsize=(15,3))

plt.plot(train_df['streams'], color='C0', label='train')
plt.plot(test_df['streams'], color='C7', label='test')
plt.plot(forecast_df['streams'], color='C1', label='forecast')
plt.legend()
plt.ylabel("Streams x 100K")

In [ ]:
show_errors(test_df['streams'],forecast_df['streams'])

### Resources

-  Fundamental time series forecasting methods in more detail[here](https://www.analyticsvidhya.com/blog/2018/02/time-series-forecasting-methods/)
-  How to use PACF to tune ARIMA models [here](https://support.minitab.com/en-us/minitab/19/help-and-how-to/statistical-modeling/time-series/how-to/partial-autocorrelation/interpret-the-results/partial-autocorrelation-function-pacf/)
